# 重新实现之前的线性热模型
应该只需要读取第一次实验的数据，这里主要想规范一下之前的写法

In [1]:
import pandas as pd
import numpy as np
import os
from keys import *
import math

from tqdm import tqdm
from loader import Loader
from thermal_model.data import *
from utils_thermal_model_raw_process import *
import matplotlib.pyplot as plt
from plotter import Plotter
import seaborn as sns

# 读取数据并且缓存成单独pickle

In [2]:
df_thermal_model_data_raw = ThermalModelData().load()

   Reading data from cache: ../.cache\thermal_model_data_1101


# 生成线性模型的输入项

In [3]:
df_thermal_model_data_raw.columns

Index(['Unnamed: 0', 'date_time', 'stack_voltage', 'stack_current', '产氢量',
       '产氢累计量', 'lye_flow', 'temp_in', 'sys_pressure', 'temp_o', 'temp_h',
       '氧侧液位', '氢侧液位', 'OTH', 'HTO', '脱氧上温', '脱氧下温', 'B塔上温', 'B塔下温', 'C塔上温',
       'C塔下温', 'A塔上温', 'A塔下温', '露点', '微氧量', '出罐压力', '进罐温度', '进罐压力', 'temp_out',
       'delta_temp', 'cell_voltage', 'current_density',
       'voltage_thermal_neutral', 'ambt_temp'],
      dtype='object')

In [4]:
def generate_model_input(df):
    df[Cols.electric_heat] = np.maximum(
        0,
        (
            df[Cols.stack_voltage] - df[Cols.voltage_thermal_neutral] * Constants.num_cells
        ) * df[Cols.stack_current] / 1000
    ) # 生成电热，如果采集到的电压信号低于理论热中性电压，则取零

    df[Cols.radiation_dissipation] = ((
            df[Cols.temp_out] + Constants.absolute_temp_delta
        ) ** 4 / 2 + (
            df[Cols.lye_temp] + Constants.absolute_temp_delta
        ) **4 / 2 - (
            df[Cols.ambt_temp] + Constants.absolute_temp_delta
        ) ** 4
    ) # 生成辐射散热，将出入口温度做平均考虑

    df[Cols.input_lye_heat] = (
        df[Cols.lye_flow] * df[Cols.lye_temp]
    ) # 跟随碱液带入的热量

    df[Cols.output_lye_heat] = (
        df[Cols.lye_flow] * df[Cols.temp_out]
    )
    
    return df

In [5]:
df_thermal_model_data_raw = generate_model_input(df_thermal_model_data_raw)

AttributeError: type object 'Cols' has no attribute 'radiation_dissapation'